In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

In [2]:
df=pd.read_csv('tel_churn.csv')
df.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


In [3]:
x=df.drop(columns=['Churn'])
y=df['Churn']

In [4]:
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7028,0,103.20,7362.90,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7029,0,29.60,346.45,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7030,1,74.40,306.60,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [5]:
y

0       0
1       0
2       1
3       0
4       1
       ..
7027    0
7028    0
7029    0
7030    1
7031    0
Name: Churn, Length: 7032, dtype: int64

In [6]:
x_train,x_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=42)

DECISION TREE

In [7]:
model_dt=DecisionTreeClassifier(criterion='entropy',random_state=100,max_depth=8,min_samples_leaf=8)

In [8]:
model_dt.fit(x_train,Y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=8, min_samples_leaf=8,
                       random_state=100)

In [9]:
Y_pred = model_dt.predict(x_test)
Y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
model_dt.score(x_test,Y_test)

0.7775408670931059

In [11]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      1033
           1       0.60      0.49      0.54       374

    accuracy                           0.78      1407
   macro avg       0.71      0.69      0.70      1407
weighted avg       0.77      0.78      0.77      1407



In [12]:
print(metrics.confusion_matrix(Y_test, Y_pred))

[[909 124]
 [189 185]]


As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [13]:
x

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,1,0,0,1,1,0,1,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,1,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,0,1,1,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,1,1,0,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,0,1,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,0,1,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
7028,0,103.20,7362.90,1,0,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,1
7029,0,29.60,346.45,1,0,0,1,0,1,1,...,0,0,1,0,1,0,0,0,0,0
7030,1,74.40,306.60,0,1,0,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [14]:
# unbalanced
y.value_counts()

Churn
0    5163
1    1869
Name: count, dtype: int64

In [15]:
# smoteenn(Synthetic Minority Over-sampling Technique) used for oversampling the minority class
sm=SMOTEENN()
X_resampled,y_resampled=sm.fit_resample(x,y)

In [16]:
# now balanced littile bit
y_resampled.value_counts()

Churn
1    3200
0    2650
Name: count, dtype: int64

In [17]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)
model_dt_smote = DecisionTreeClassifier(criterion='gini',random_state=10,max_depth=6,min_samples_leaf=8)

In [18]:
import pickle
with open('Churn_model_columns.pkl', 'wb') as f:
    pickle.dump(xr_train.columns.tolist(), f)

In [19]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.935042735042735
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       544
           1       0.92      0.96      0.94       626

    accuracy                           0.94      1170
   macro avg       0.94      0.93      0.93      1170
weighted avg       0.94      0.94      0.93      1170



In [20]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[493  51]
 [ 25 601]]


Now we can see quite better results, i.e. Accuracy: 94 %, and a very good recall, precision & f1 score for minority class.

Let's try with some other classifier.

Random Forest Classifier

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
model_rf = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=100,max_depth=6,min_samples_leaf=8)

In [23]:
model_rf.fit(x_train,Y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [24]:
y_pred = model_rf.predict(x_test)
print(model_rf.score(x_test,Y_test))
print(classification_report(Y_test,y_pred))

0.7789623312011372
              precision    recall  f1-score   support

           0       0.81      0.91      0.86      1033
           1       0.63      0.42      0.50       374

    accuracy                           0.78      1407
   macro avg       0.72      0.66      0.68      1407
weighted avg       0.76      0.78      0.76      1407



Again precision , recall and f1-score are less for low minority class labeled as 1, so using resampled data with the help of smoteenn

In [25]:
model_rf_smote =  RandomForestClassifier(n_estimators=100,criterion='gini',random_state=100,max_depth=6,min_samples_leaf=8)
model_rf_smote.fit(xr_train,yr_train)
smote_rf_pred=model_rf_smote.predict(xr_test)
print(model_rf_smote.score(xr_test,yr_test))
print(classification_report(smote_rf_pred,yr_test))

0.9273504273504274
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       487
           1       0.98      0.90      0.94       683

    accuracy                           0.93      1170
   macro avg       0.92      0.93      0.93      1170
weighted avg       0.93      0.93      0.93      1170



In [26]:
import pickle
filename = "Churn_predictor.sav"
with open(filename,'wb') as file:
    pickle.dump(model_rf_smote,file)

In [27]:
model = pickle.load(open(filename,'rb'))

In [34]:
xr_test.iloc[0]

SeniorCitizen                                0.000000
MonthlyCharges                              53.148256
TotalCharges                               257.673175
gender_Female                                1.000000
gender_Male                                  0.000000
Partner_No                                   1.000000
Partner_Yes                                  0.000000
Dependents_No                                1.000000
Dependents_Yes                               0.000000
PhoneService_No                              0.000000
PhoneService_Yes                             1.000000
MultipleLines_No                             0.000000
MultipleLines_No phone service               0.000000
MultipleLines_Yes                            0.000000
InternetService_DSL                          1.000000
InternetService_Fiber optic                  0.000000
InternetService_No                           0.000000
OnlineSecurity_No                            0.000000
OnlineSecurity_No internet s

In [28]:
model.score(xr_test,yr_test)

0.9273504273504274